In [1]:
import pandas as pd
import numpy as np
#from tqdm import tqdm_notebook, trange
from tqdm.notebook import trange, tqdm

import itertools
import os
import re
import pickle
import json
import pathlib
import copy


import torch
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from torch_geometric.utils import dropout_edge, mask_feature

from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, f1_score
from sklearn.utils import class_weight
from sklearn.model_selection import StratifiedKFold

# visual
import matplotlib.pyplot as plt

#local
from GCN_model.Standard_GCN import GCN

# Cuda preparation

In [2]:
if torch.cuda.is_available():
    device = torch.device('cpu')
    #device = torch.device('cuda')
    #CUDA_LAUNCH_BLOCKING=1
else:
    device = torch.device('cpu')

#device = torch.device('cpu')
print(f'running with {device}')

running with cpu


# Hypertuning + training logic

In [3]:
def outptut_variance_compute(model, graphs_dataset):
    cumul_variance = 0
    for batchdata in graphs_dataset:
            batchdata_list = batchdata.to_data_list()
            for data in batchdata_list:
                data.to(device)
                output = model(data.x, data.edge_index)
                pred = output.argmax(dim=1)

                cumul_variance += np.var(pred.detach().cpu().numpy())
                
    return str(cumul_variance)


        

In [4]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [ ]:
def model_training(graphs_dataset, num_classes, params):

    best_accuracy = -1
    best_F1 = -1
    best_params = None
    best_model_dict = None
    best_cm = None
    best_f1 = None
    best_cm = None
    best_roc_auc = -1
    len_graph_dataset = len(graphs_dataset)
    
    # creating weights
    labels = []
    for data in graphs_dataset:
        labels.extend(data.y.numpy())

    labels = np.array(labels)

    #### if weights desired ####
    #class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(labels), y=labels)
    #class_weights = torch.tensor(class_weights, dtype=torch.float64).to(device)
    
    for batch in graphs_dataset:
        first_batch = batch
        first_graph = first_batch[0]
        break

    val_losses_list = []
    train_losses_list = []
    val_acc_list = []
    train_acc_list = []
    test_acc_list = []

    batch_size, max_epochs, num_layers, hidden_channels, optimizer_type, learning_rate, dropout_rate, fc_layer_channels = params

    model = GCN(num_features=first_graph.num_features, hidden_channels=hidden_channels, num_classes=num_classes, num_layers=num_layers, dropout_rate=dropout_rate, fc_layer_channels=fc_layer_channels).double().to(device)

    if optimizer_type == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)
    else:
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=5e-4)

    best_model_state_dict = None
    best_validation_acc = -1
    best_epoch = -1
    #early_stopper = EarlyStopper(patience=100, min_delta=0)
    
    #for epoch in range(max_epochs):
    for epoch in trange(max_epochs, leave=False):
        model.train()
        total_loss = 0
        total_correct = 0
        total_nodes = 0

        for batch in graphs_dataset:
            batch.train_mask = batch.train_mask.type(torch.bool)

            data = batch.to(device)
            
            #this does xero_grad() more efficiently
            for param in model.parameters():
                param.grad = None

            output = model(data.x, data.edge_index)
            pred = output.argmax(dim=1)

            #correct = pred.eq(data.y).sum().item()
            correct = (pred[data.train_mask] == data.y[data.train_mask]).sum()
            total_correct += correct
            total_nodes += int(torch.sum(data.train_mask))

            loss = F.cross_entropy(output[data.train_mask], data.y[data.train_mask])#weight=class_weights
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        train_losses_list.append(total_loss / len_graph_dataset)
        train_acc = total_correct / total_nodes
        train_acc_list.append(train_acc.detach().cpu().numpy())

        # Validation after each epoch on the validation set (val_mask)
        model.eval()
        total_correct_val = 0
        total_nodes_val = 0
        total_val_loss = 0

        with torch.no_grad():
            for batch in graphs_dataset:
                batch.val_mask = batch.val_mask.type(torch.bool)
                data = batch.to(device)

                output = model(data.x, data.edge_index)
                pred = output.argmax(dim=1)
                correct = (pred[data.val_mask] == data.y[data.val_mask]).sum()

                total_correct_val += correct
                total_nodes_val += int(torch.sum(data.val_mask))
                val_loss = F.cross_entropy(output[data.val_mask], data.y[data.val_mask])#weight=class_weights
                if not np.isnan(val_loss.item()):
                    total_val_loss += val_loss.item()
        
        if np.isnan(total_val_loss / len_graph_dataset):
            print(f'NaN detected: {total_val_loss} and {len_graph_dataset}')
        val_losses_list.append(total_val_loss / len_graph_dataset)
        val_acc = total_correct_val / total_nodes_val
        val_acc_list.append(val_acc.detach().cpu().numpy())

        #if early_stopper.early_stop(total_val_loss / len_graph_dataset):             
            #break
        
        if val_acc > best_validation_acc:
                best_validation_acc = val_acc
                best_epoch = epoch
                best_model_state_dict = model.state_dict().copy()
        
    # Use the test_mask for final evaluation
    if best_model_state_dict is not None:
        # Load the best model state back into the model
        model.load_state_dict(best_model_state_dict)

        model.eval()
        total_correct = 0
        total_nodes = 0
        true_labels = list()
        predicted_labels = list()

        with torch.no_grad():
            for batch in graphs_dataset:
                batch.test_mask = batch.test_mask.type(torch.bool)
                data = batch.to(device)

                output = model(data.x, data.edge_index)
                pred = output.argmax(dim=1)
                correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
                total_correct += correct
                total_nodes += int(torch.sum(data.test_mask))
                true_labels.extend(data.y[data.test_mask].cpu().numpy())
                predicted_labels.extend(pred[data.test_mask].cpu().numpy())


        test_acc_best = total_correct / total_nodes
        test_acc_list.append(test_acc_best.detach().cpu().numpy())
        
        cm = confusion_matrix(true_labels, predicted_labels)

        roc_auc = roc_auc_score(true_labels, predicted_labels)

        cumul_variance = outptut_variance_compute(model, graphs_dataset)

    if test_acc_best > best_accuracy:
        best_accuracy = test_acc_best
        best_params = params
        best_cm = cm
        best_roc_auc = roc_auc
        best_model_dict = model.state_dict().copy()

    window_size = 10
    val_losses_list_mv = np.convolve(val_losses_list, np.ones(window_size) / window_size, mode='valid').tolist()
    train_acc_losses_list_mv = np.convolve(train_losses_list, np.ones(window_size) / window_size, mode='valid').tolist()
    train_acc_list_mv = np.convolve(train_acc_list, np.ones(window_size) / window_size, mode='valid').tolist()
    

    plot_dict = {'val_losses_list': val_losses_list_mv,
                    'train_losses_list': train_acc_losses_list_mv,
                    'val_acc_list': val_acc_list,
                    'train_acc_list': train_acc_list_mv,
                    'test_acc_list': test_acc_list}

    return best_accuracy, best_params, best_model_dict, best_cm, best_roc_auc, plot_dict, cumul_variance

# Results saving tools

In [6]:
def create_dir_if_absent(path):
    if not os.path.exists(path):      
        os.makedirs(path)

In [7]:
# Makes sure the output folder that contains every result subfolders exists at ../Output/Trained_models 

if not os.path.exists("../../Output/Trained_models_masking"):      
    # if the demo_folder directory is not present 
    # then create it.
    os.makedirs("../../Output/Trained_models_masking")

In [8]:
# Creates a result sub-folder for this run
def create_next_model_folder(directory_path):

    folders = [f for f in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, f))]

    largest_number = -1

    # Iterate through the folders
    for folder in folders:
        # Use regular expression to extract the number at the end of the folder name
        match = re.search(r'\d+$', folder)
        if match:
            number = int(match.group())
            if number > largest_number:
                largest_number = number

    os.makedirs(f"../../Output/Trained_models_masking/experiment_{largest_number+1}")
    print(f'created folder ../../Output/Trained_models_masking/experiment_{largest_number+1}')
    return (f'../../Output/Trained_models_masking/experiment_{largest_number+1}')

In [9]:
def create_result_df(directory, runs):
    subdirectories = [subdir for subdir in os.listdir(directory)]

    row_names = list(set(subdirectories))

    col_names = [i for i in range(runs)]

    df = pd.DataFrame(columns=col_names, index=row_names)

    return (df) 

# Main loop

In [10]:
def count_unique_datasets(project):
    dataset_count = 0
    for dataset in project:
        folder_list = os.listdir(dataset)
        dataset_count += len(folder_list)
    
    return dataset_count

In [11]:
# Check if a row exist in the df that already cover a specific instance
def row_exists(df, new_row):

    return ((df['crop'] == str(new_row['crop'])) 
            & (df['dataset'] == str(new_row['dataset'])) 
            & (df['augmentation'] == str(new_row['augmentation'])) 
            & (df['run_number'] == int(new_row['run_number']))
            & (df['loop'] == int(new_row['loop']))
            & (df['parameters'] == str(new_row['parameters']))).any()

In [12]:
def add_row(df, new_row):
    if not row_exists(df, new_row):
        # Get the intersection of keys in new_row and columns of DataFrame
        valid_keys = set(new_row.keys()) & set(df.columns)
        
        # Create a new dictionary with only valid keys
        valid_new_row = {key: new_row[key] for key in valid_keys}

        valid_new_row_df = pd.DataFrame([valid_new_row])

        # Concatenate df and valid_new_row_df
        df = pd.concat([df, valid_new_row_df], ignore_index=True)
    return df

|output
-|analysis_i
--|datasets_dir
---|dataset_folder
----|params
-----|models

In [13]:
def create_analysis_config_file(path, name, config_dict):
                                
    file_path = pathlib.Path(path) / name
    with file_path.open('w') as f:
        json.dump(config_dict, f, indent=4)

In [14]:
def generate_analysis_config(**kwargs):
    config_dict = {
        **kwargs
    }
    return config_dict

In [15]:
def build_result_dir_and_files(config_dict, col_list):
    # Create the analysis subfolder that will contains the trained models and accuracy results
    if config_dict["restart"] is None:
        trained_models_path = create_next_model_folder('../../Output/Trained_models_masking')
        create_analysis_config_file(trained_models_path, "analysis_config.json", config_dict)
    else:
        trained_models_path = config_dict["restart"]



    if os.path.exists(f'{trained_models_path}/summary.csv'):
        result_df = pd.read_csv(f'{trained_models_path}/summary.csv', header=0, index_col=0)

    else:
        result_df_cols = ['crop', 'dataset', 'augmentation', 'loop', 'run_number', 'parameters', 'accuracy', 'TP', 'FP', 'FN', 'TN', 'ROC-AUC', 'precision', 'recall', 'F1-score', 'epochs', 'cumul_variance']
        result_df_cols += list(col_list)
        result_df = pd.DataFrame(columns=result_df_cols)
    
    return trained_models_path, result_df

In [16]:
def add_subplot(axes, kfold_counter, plot_dict, best_accuracy, cm):
    axes[kfold_counter, 0].plot(range(len(plot_dict['train_losses_list'])), plot_dict['train_losses_list'], label='Training Loss')
    axes[kfold_counter, 0].plot(range(len(plot_dict['val_losses_list'])), plot_dict['val_losses_list'], label='Validation Loss')
    axes[kfold_counter, 0].set_xlabel('Epochs')
    axes[kfold_counter, 0].set_ylabel('Loss')
    axes[kfold_counter, 0].legend()

    axes[kfold_counter, 1].plot(range(len(plot_dict['train_acc_list'])), plot_dict['train_acc_list'], label='Training Accuracy')
    axes[kfold_counter, 1].plot(range(len(plot_dict['val_acc_list'])), plot_dict['val_acc_list'], label='Validation Accuracy')
    axes[kfold_counter, 1].plot(range(len(plot_dict['test_acc_list'])), plot_dict['test_acc_list'], label='Test Accuracy')
    axes[kfold_counter, 1].axhline(y = round(float(best_accuracy.cpu()), 2), color = 'r', linestyle = '-')

    max_acc_train = float(max(plot_dict['train_acc_list']))
    max_acc_train_idx = int(np.argmax(plot_dict['train_acc_list']))
    axes[kfold_counter, 1].annotate(f'Highest train Acc: {round(max_acc_train, 2)}', 
            xy=(max_acc_train_idx, max_acc_train),
            xytext=(max_acc_train_idx, max_acc_train -0.15),  # Adjust text position
            arrowprops=dict(facecolor='red', arrowstyle='->'))
    
    max_valid_acc = float(max(plot_dict['val_acc_list']))
    max_valid_acc_idx = int(np.argmax(plot_dict['val_acc_list']))
    axes[kfold_counter, 1].annotate(f'Highest Val Acc: {round(max_valid_acc, 2)}', 
            xy=(max_valid_acc_idx, max_valid_acc),
            xytext=(max_valid_acc_idx, max_valid_acc -0.15),  # Adjust text position
            arrowprops=dict(facecolor='red', arrowstyle='->'))

    axes[kfold_counter, 1].set_xlabel('Epochs')
    axes[kfold_counter, 1].set_ylabel('Accuracy')
    axes[kfold_counter, 1].legend()

    axes[kfold_counter, 2].imshow(cm, cmap='Blues')
    for i in range(len(cm)):
        for j in range(len(cm)):
            axes[kfold_counter, 2].text(j, i, f'{cm[i, j]}', ha='center', va='center', color='black', fontsize=12)
    axes[kfold_counter, 2].set_xticks(np.arange(len(cm)))
    axes[kfold_counter, 2].set_yticks(np.arange(len(cm)))
    axes[kfold_counter, 2].set_xlabel('Predicted label')
    axes[kfold_counter, 2].set_ylabel('True label')
    axes[kfold_counter, 2].set_title('Confusion Matrix Heatmap')
    axes[kfold_counter, 2].axis('off')

In [17]:
def kfold_reset_train_valid_masks (graphs_dataset):
    graphs_list = []
    train_mask_list = []
    val_mask_list = []
    plant_tags_list = []
    label_list = []
    for batchdata in graphs_dataset:
        batchdata_list = batchdata.to_data_list()
        for data in batchdata_list:
            graphs_list.append(data)
            for i in data.train_mask:
                train_mask_list.append(int(i))
            for i in data.val_mask:
                val_mask_list.append(int(i))
            for i in data.plant_tags:
                plant_tags_list.append(int(i))
            for i in data.y:
                label_list.append(int(i))
    
    train_mask_array = np.array(train_mask_list)
    val_mask_array = np.array(val_mask_list)
    plant_tags_array = np.array(plant_tags_list)
    label_array = np.array(label_list)

    combined_mask = np.logical_or(train_mask_array, val_mask_array)
    num_ones = np.sum(combined_mask)
    num_zeros = len(combined_mask) - num_ones

    filtered_plant_tags = plant_tags_array[combined_mask]
    filtered_labels = label_array[combined_mask]

    return filtered_plant_tags, filtered_labels, graphs_list


In [18]:
def shift_kfold_mask(filtered_plant_tags, train_idx, valid_idx, graphs_list):
    train_tags = [filtered_plant_tags[i] for i in train_idx]
    valid_tags = [filtered_plant_tags[i] for i in valid_idx]

    #assigning new masks
    for graph in graphs_list:
        #create new masks array
        new_train_mask = np.isin(graph.plant_tags, train_tags)
        new_val_mask = np.isin(graph.plant_tags, valid_tags)
        new_test_mask = np.logical_and(~new_train_mask, ~new_val_mask)

        graph.train_maks = torch.Tensor(new_train_mask)
        graph.val_maks = torch.Tensor(new_val_mask)
        graph.test_maks = torch.Tensor(new_test_mask)
    
    return graphs_list

In [19]:
def dropout_edges_for_graphs(graphs_list, p, force_undirected=True):
    
    for data in graphs_list:

        new_edge_index, edge_mask = dropout_edge(data.edge_index, p=p, force_undirected=force_undirected)
        data.edge_index = new_edge_index
    
    return graphs_list


In [20]:
def mask_features_for_graphs(graphs_list, p):
    
    for data in graphs_list:

        x, feat_mask = mask_feature(data.x, p=p, mode ='all', fill_value=0)
        data.x = x
    
    return graphs_list

In [21]:
def extract_crop_name(input_string):
     # Define the possible crop names
    onion_list = ["oignon", "onion", "oignons", "onions"]
    carrot_list = ["carrot", "carrots", "carotte", "carottes"]
    lettuce_list = ["lettuce", "letucces", "laitue", 'laitues']

    # Track matches for each list
    matches = {
        "onion": any(word in input_string for word in onion_list),
        "carrot": any(word in input_string for word in carrot_list),
        "lettuce": any(word in input_string for word in lettuce_list)
    }

    # Count the number of lists with matches
    match_count = sum(matches.values())

    if match_count == 1:
        for key, matched in matches.items():
            if matched:
                return f"{key}"
    elif match_count > 1:
        return "Unknown"
    else:
        return "Unknown"

In [ ]:
analysis_params= {
    "project": ['graph_datasets/onion/onion_base'],
    "classes": 2,
    "loops": 5,
    "kfold": 5,
    "start_seed": 0,
    "restart": None
}

#store search parameters here
param_grid = { 
    'batch_size' : [8, 16], #keep batch size first
    'max_epochs': [2000],
    'num_layers': [3, 4, 5],
    'hidden_channels': [32, 34],
    'optimizer_type': ['Adam'], 
    'learning_rate': [0.00001],
    'dropout_rate': [0, 0.1],
    'fc_layer_channels' : [0]
    }

# create result path, 
config_dict = generate_analysis_config(**analysis_params, **param_grid )

param_dict = dict()
param_folder= None

#create ouput path, save analysis config, and create result_df
trained_models_path, result_df = build_result_dir_and_files(config_dict, param_grid)

dataset_count = count_unique_datasets(config_dict["project"])
param_combi = len(list(itertools.product(*param_grid.values())))

with tqdm(total=dataset_count*config_dict["kfold"]*param_combi*config_dict["loops"]) as pbar: 
    for loop in range(config_dict["loops"]):
        print(f'LOOP {loop+1}/{config_dict["loops"]} STARTING')

        for datasets_dir in config_dict["project"]:
            dataset_list = os.listdir(datasets_dir)
            for dataset_folder in dataset_list:
                dataset = dataset_folder.split('_', 1)[0]
                aug = dataset_folder.split('_', 1)[1]
                graphs_dataset = torch.load(f'{datasets_dir}/{dataset_folder}/graphs_dataset.pth')

                param_combinations = list(itertools.product(*param_grid.values()))

                for param in param_combinations:
                    active_row = False
                    param_folder = f'params_{param}' #loop

                    json_path = pathlib.Path(f'{trained_models_path}/{datasets_dir}/{dataset_folder}/{param_folder}')

                    # Check if the file exists and is a file
                    if json_path.exists() and json_path.is_file():
                        pass
                    else:
                        #extract the first graph to find out the num of features in the graph dataset
                        for batch in graphs_dataset:
                            first_batch = batch
                            num_features = first_batch[0].num_features
                            break

                        run_param_dict = dict(map(lambda i,j : (i,j) , param_grid.keys(), param))
                        run_param_dict['crop'] = extract_crop_name(str(datasets_dir))
                        run_param_dict['dataset'] = dataset
                        run_param_dict['augmentation'] = aug
                        run_param_dict['num_features'] = num_features
                        json_path.mkdir(parents=True, exist_ok=True)
                        create_analysis_config_file(json_path, 'model_config.json', run_param_dict)
                        


                    plt.close() #avoids opening multiple figures for nothing when restarting an analysis
                    fig, axes = plt.subplots(nrows = config_dict["kfold"], ncols =3, figsize=(15, 3*config_dict["kfold"]))
                    plt.suptitle(f'dataset: {dataset} \n augmentation: {aug} \n params: {param}')

                    #####################
                    ### kfold start  ####
                    #####################
                    kfold_counter = -1
                    seed = config_dict["start_seed"]
            
                    filtered_plant_tags, filtered_labels, complete_graphs_list = kfold_reset_train_valid_masks(graphs_dataset)

                    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state= seed)
                    seed += 1

                    for train_idx, valid_idx in skf.split(filtered_plant_tags, filtered_labels):

                        graphs_list = shift_kfold_mask(filtered_plant_tags, train_idx, valid_idx, complete_graphs_list)
                        modified_graphs_list = copy.deepcopy(graphs_list)

                        graphs_dataset_kfold = DataLoader(modified_graphs_list, batch_size=param[0], shuffle=True) #pin_memory=True
                        kfold_counter += 1

                        new_row = {
                        'crop': datasets_dir, 
                        'dataset': dataset, 
                        'augmentation': aug,
                        'loop': loop,
                        'run_number': kfold_counter,
                        'parameters': param
                        }

                        #print(f'{row_exists(result_df, new_row)}')
                        if not row_exists(result_df, new_row):
                            active_row = True
                            pbar.set_description(f"Processing {dataset_folder} - run {kfold_counter}")
                            best_accuracy, params, model_state_dict, cm, roc_auc, plot_dict, cumul_variance = model_training(graphs_dataset = graphs_dataset_kfold,
                                                                                    num_classes=config_dict["classes"], 
                                                                                    params=param)

                            params_values_dict = dict(map(lambda i,j : (i,j) , param_grid.keys(), param))
                            model_number = (loop*config_dict['kfold']) + kfold_counter
                            torch.save(model_state_dict, f'{trained_models_path}/{datasets_dir}/{dataset_folder}/{param_folder}/{dataset}_{model_number}.pt')

                            if (cm[0, 0] + cm[0, 1]) == 0:
                                precision = 0
                            else:
                                precision = float(cm[0, 0]/(cm[0, 0] + cm[0, 1])) # TP/(TP + FP)
                            
                            if (cm[0, 0] + cm[1, 0]) == 0:
                                rappel = 0
                            else:
                                recall = float(cm[0, 0]/(cm[0, 0] + cm[1, 0])) # TP/(TP+FN)
                            
                            if precision > 0 and recall > 0:
                                f1_score = float((2 * precision * recall)/(precision + recall))
                            else:
                                f1_score = 0

                            result_dict= {
                                'accuracy': float(best_accuracy*100), 
                                'TP': cm[0, 0], 
                                'FP':cm[0, 1], 
                                'FN':cm[1, 0], 
                                'TN':cm[1, 1], 
                                'ROC-AUC': float(roc_auc),
                                'precision': precision,
                                'recall': recall,
                                'F1-score': f1_score,
                                'epochs': len(plot_dict['train_losses_list']),
                                'cumul_variance' : cumul_variance
                            }

                            #merge 3 dicts to complete the row in the result_df
                            new_row.update(result_dict)
                            new_row.update(params_values_dict)

                            result_df = add_row(result_df, new_row)
                            result_df.to_csv(f'{trained_models_path}/summary.csv')


                            # Add the new plots to the existing subplots
                            add_subplot(axes, kfold_counter, plot_dict, best_accuracy, cm)

                            # param_folder only None if retracing already done runs
                            if param_folder is not None:
                                plt.savefig(f'{trained_models_path}/{datasets_dir}/{dataset_folder}/{param_folder}{dataset_folder}_loop_{loop}_run{kfold_counter}.svg')
                            
                        pbar.update(1)
                        
                    # param_folder only None if retracing already done runs
                    if param_folder is not None and active_row:   
                        plt.savefig(f'{trained_models_path}/{datasets_dir}/{dataset_folder}/{param_folder}/{dataset_folder}_loop_{loop}_summary.svg')
                        plt.close(fig)

created folder ../../Output/Trained_models_masking/experiment_27


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'models_result_replication/onion_base'